In [145]:
import folium
import pandas as pd
import json
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [146]:
manfred = pd.read_csv("/home/bosser/Documentos/PROYECTOFINAL/CSV/CSV_manfred/resultado_presencial.csv")
manfred

,id,Presencial
0,id_man_1,MADRID
1,id_man_2,BARCELONA
2,id_man_3,MADRID
3,id_man_4,MADRID
4,id_man_5,MADRID
...,...,...
1258,id_man_1222,NaN
1259,id_man_1223,MADRID
1260,id_man_1224,MADRID
1261,id_man_1225,NaN


In [147]:
manfred_cuenta = manfred.groupby("Presencial", as_index= False).size()
manfred_cuenta["Presencial"] = manfred_cuenta["Presencial"].str.title()

manfred_cuenta  

,Presencial,size
0,A Coruña,44
1,Alcobendas,1
2,Alcoy,1
3,Alicante (Alacant),2
4,Badajoz,1
5,Barcelona,73
6,Bergondo,1
7,Bilbao,5
8,"Ciudad De México, México",1
9,Gijón,2


In [148]:
from geopy.geocoders import Nominatim
df= manfred_cuenta

# Crear un geolocalizador con un user_agent personalizado
geolocator = Nominatim(user_agent="tu_aplicacion_que_no_abusa_de_servicio")

def obtener_geolocalizacion(ciudad):
    try:
        location = geolocator.geocode(ciudad, timeout=10)
        if location:
            address = location.raw.get("display_name", "").split(", ")
            pais = address[-1]  # Último elemento es el país
            provincia = address[-3] if len(address) > 2 else None  # Provincia (en muchos casos)
            return pd.Series([pais, provincia])
    except GeocoderTimedOut:
        return pd.Series([None, None])

df[["País", "Provincia"]] = df["Presencial"].apply(obtener_geolocalizacion)



In [149]:
df

,Presencial,size,País,Provincia
0,A Coruña,44,España,A Coruña
1,Alcobendas,1,España,Alcobendas
2,Alcoy,1,España,Alacant / Alicante
3,Alicante (Alacant),2,España,Alacant / Alicante
4,Badajoz,1,España,Badajoz
5,Barcelona,73,España,Barcelona
6,Bergondo,1,España,A Coruña
7,Bilbao,5,España,Bizkaia
8,"Ciudad De México, México",1,México,Cuauhtémoc
9,Gijón,2,España,Gijón / Xixón


In [150]:
df_españa = df[df["País"] == "España"]
           
# Agruparmos por provincia y sumamos
provincia_size_sum = df_españa.groupby("Provincia", as_index= False)["size"].sum()

# Mostrar el resultado
provincia_size_sum

,Provincia,size
0,A Coruña,45
1,Alacant / Alicante,3
2,Alcobendas,1
3,Araba/Álava,2
4,Badajoz,1
5,Barcelona,73
6,Bizkaia,5
7,Gijón / Xixón,2
8,Gipuzkoa,1
9,Granada,1


In [151]:
print(provincia_size_sum["Provincia"])

0                         A Coruña
1               Alacant / Alicante
2                       Alcobendas
3                      Araba/Álava
4                          Badajoz
5                        Barcelona
6                          Bizkaia
7                    Gijón / Xixón
8                         Gipuzkoa
9                          Granada
10                            Jaén
11                      Las Palmas
12                            León
13                          Lleida
14                         Logroño
15                          Madrid
16                          Málaga
17                          Oviedo
18                           Palma
19                  Pamplona/Iruña
20                      Pontevedra
21                       Santander
22                         Sevilla
23                      Valladolid
24             València / Valencia
25                        Zaragoza
26    Área Metropolitana de Murcia
Name: Provincia, dtype: object


In [152]:
import geopandas as gpd
import folium
from folium.plugins import HeatMap


geojson_path = "/home/bosser/Documentos/PROYECTOFINAL/Graficas/data/spain-provinces.geojson"

# Cargar archivo GeoJson
gdf_provincias = gpd.read_file(geojson_path)

#Renonbramos
df_españa["Provincia"] = df_españa["Provincia"].replace({
    "A Coruña": "A Coruña",
    "Alacant / Alicante": "Alacant/Alicante",
    "Araba/Álava": "Araba/Álava",
    "Badajoz": "Badajoz",
    "Barcelona": "Barcelona",
    "Granada": "Granada",
    "Jaén": "Jaén",
    "Las Palmas": "Las Palmas",
    "Palma" : "Illes Balears",
    "León": "León",
    "Lleida": "Lleida",
    "Madrid": "Madrid",
    "Málaga": "Málaga",
    "Pontevedra": "Pontevedra",
    "Sevilla": "Sevilla",
    "Valladolid": "Valladolid",
    "València / Valencia": "València/Valencia",
    "Zaragoza": "Zaragoza",
    "Área Metropolitana de Murcia": "Murcia",
    "Alcobendas" : "Madrid",
    "Bizkaia":"Bizkaia/Vizcaya",
    "Gijón / Xixón": "Asturias",
    "Oviedo": "Asturias",
    "Gipuzkoa": "Gipuzkoa/Guipúzcoa",
    "Logroño" : "La Rioja"
})

#Unimos datos de provincia
gdf_merged = gdf_provincias.set_index("name").join(df_españa.set_index("Provincia"))
gdf_merged = gdf_merged.reset_index()
gdf_merged = gdf_merged.drop(columns=["created_at", "updated_at", "cartodb_id"])


#Crear maoa de españa
m = folium.Map(location=[40.4637, -3.7492], zoom_start=6)  # Coordenadas de España

#Añadimos los datos para hacerlo Cloropletico
folium.Choropleth(
    geo_data=gdf_merged,
    name="choropleth",
    data=gdf_merged,
    columns=["name", "size"],  
    key_on="feature.properties.name", 
    fill_color="YlOrRd", 
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name="Tamaño por Provincia",
    nan_fill_color= "gray",
    nan_fill_opacity= 0.5
).add_to(m)

#Lo guardamos en html para subirlo a streamlit
m.save("mapa_cloropletico_espana.html")  # Guarda el mapa como archivo HTML

/tmp/ipykernel_5937/3930094343.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_españa["Provincia"] = df_españa["Provincia"].replace({


In [153]:
print(list(gdf_provincias["name"]))

['Illes Balears', 'Asturias', 'A Coruña', 'Girona', 'Las Palmas', 'Pontevedra', 'Santa Cruz De Tenerife', 'Cantabria', 'Málaga', 'Almería', 'Murcia', 'Albacete', 'Ávila', 'Araba/Álava', 'Badajoz', 'Alacant/Alicante', 'Ourense', 'Barcelona', 'Burgos', 'Cáceres', 'Cádiz', 'Castelló/Castellón', 'Ciudad Real', 'Jaén', 'Córdoba', 'Cuenca', 'Granada', 'Guadalajara', 'Gipuzkoa/Guipúzcoa', 'Huelva', 'Huesca', 'León', 'Lleida', 'La Rioja', 'Soria', 'Navarra', 'Ceuta', 'Lugo', 'Madrid', 'Palencia', 'Salamanca', 'Segovia', 'Sevilla', 'Toledo', 'Tarragona', 'Teruel', 'València/Valencia', 'Valladolid', 'Bizkaia/Vizcaya', 'Zamora', 'Zaragoza', 'Melilla']


In [154]:
print(df_españa.columns)


Index(['Presencial', 'size', 'País', 'Provincia'], dtype='object')


In [155]:
gdf_merged

,name,cod_prov,cod_ccaa,geometry,Presencial,size,País
0,Illes Balears,07,03,"MULTIPOLYGON (((3.21364 39.95751, 3.1544 39.92...",Palma,5.0,España
1,Asturias,33,18,"MULTIPOLYGON (((-5.84083 43.66062, -5.83002 43...",Gijón,2.0,España
2,Asturias,33,18,"MULTIPOLYGON (((-5.84083 43.66062, -5.83002 43...",Oviedo,1.0,España
3,A Coruña,15,11,"MULTIPOLYGON (((-7.6803 43.78714, -7.66231 43....",A Coruña,44.0,España
4,A Coruña,15,11,"MULTIPOLYGON (((-7.6803 43.78714, -7.66231 43....",Bergondo,1.0,España
5,Girona,17,08,"MULTIPOLYGON (((1.74871 42.49482, 1.82355 42.4...",NaN,NaN,NaN
6,Las Palmas,35,04,"MULTIPOLYGON (((-13.89051 28.75685, -13.83632 ...",Las Palmas De Gran Canaria,1.0,España
7,Pontevedra,36,11,"MULTIPOLYGON (((-8.18101 42.85828, -8.11637 42...",Vigo,6.0,España
8,Santa Cruz De Tenerife,38,04,"MULTIPOLYGON (((-16.15754 28.58816, -16.12669 ...",NaN,NaN,NaN
9,Cantabria,39,05,"MULTIPOLYGON (((-3.59024 43.51356, -3.54634 43...",NaN,NaN,NaN
